# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Импортируем библиотеки, которые пригодятся в исследовании:

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import random
import time

## Откройте и изучите файл

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(data)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


**сalls** — количество звонков

**minutes** — суммарная длительность звонков в минутах

**messages** — количество sms-сообщений

**mb_used** — израсходованный интернет-трафик в Мб

**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

Соответственно логика в данном датафрейме такая,  что при **is_ultra** = 1 - клиенту подойдёт тариф ultra исходя из признаков в остальных столбцах, при **is_ultra** = 0, вероятнее всего, - smart

## Разбейте данные на выборки

В теоретической части не нашел такого, но вроде нагуглил как с привлечением библиотеки numpy можно перемешать данные и потом разделить их сразу на 3 категории. Соответственно, пусть 60 % - будут обучающей выборкой, 20 % - валидационной, а 20 % - тестовой

In [4]:
train, validate, test = np.split(data.sample(frac=1, random_state=42), [int(0.6*len(data)), int(0.8*len(data))])

In [5]:
display(train)

,calls,minutes,messages,mb_used,is_ultra
506,46.0,338.60,35.0,11428.54,0
2513,39.0,242.71,0.0,20480.11,0
354,39.0,258.02,0.0,19998.80,0
1080,36.0,230.99,19.0,23525.07,1
2389,35.0,205.35,52.0,35177.94,1
...,...,...,...,...,...
3177,114.0,731.76,10.0,25311.22,0
2519,57.0,454.05,13.0,17220.68,0
1508,80.0,574.98,75.0,23750.99,0
1750,28.0,169.93,49.0,21232.14,0


In [6]:
display(validate)

,calls,minutes,messages,mb_used,is_ultra
660,11.0,66.00,0.0,7039.12,0
2878,90.0,738.58,8.0,22070.76,0
338,55.0,363.71,31.0,16303.01,0
215,35.0,267.67,25.0,18231.83,0
1615,91.0,643.90,103.0,22197.84,0
...,...,...,...,...,...
79,43.0,303.51,37.0,15063.41,0
1172,144.0,1031.79,99.0,40174.34,1
814,114.0,793.69,15.0,23225.78,0
1852,74.0,563.59,30.0,10781.76,0


In [7]:
display(test)

,calls,minutes,messages,mb_used,is_ultra
2817,12.0,86.62,22.0,36628.85,1
2105,102.0,760.64,0.0,23166.00,1
133,109.0,704.30,79.0,10603.12,0
1140,40.0,195.79,55.0,19493.38,0
933,2.0,2.00,28.0,9173.78,0
...,...,...,...,...,...
1095,62.0,454.02,35.0,15018.46,0
1130,69.0,465.96,12.0,14982.27,0
1294,40.0,280.44,2.0,13934.54,0
860,72.0,410.23,68.0,16006.55,0


Вроде бы выглядит вполне успешно. Теперь разобьём каждую из этих выборок на признаки features и целевой признак target

In [8]:
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']
features_validate = validate.drop(['is_ultra'], axis=1)
target_validate = validate['is_ultra']
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

## Исследуйте модели

Для задач классификации используются решающее дерево (быстрая скорость обучения, но низкое качество), случайный лес (высокое качество, но низкая скорость обучения), логистическая регрессия(средняя качество ответов, но высокая скорость обучения). Попробуем обучения при помощи всех этих трёх моделей и выберем лучшую

### Решающее дерево

У решающего дерева есть 3 гиперпараметра, которые были рассмотрены: глубина дерева, которая не должна быть слишком большой для избежания переобучения, но и не слишком малой для недообучения; min_samples_split - минимальное кол-во выборок для разделения внутреннего узла; min_samples_leaf - минимальное кол-во образцов в листовом узле

Напишем цикл, который прогонит каждый из этих параметров в определённом диапазоне и найдём их наилучшую комбинацию. Так как решающее дерево самая быстрообучаемая модель, вроде бы не особо нагружающая систему, позволим себе сделать диапазоны достаточно большими:

In [9]:
best_tree_accuracy = 0
best_tree_depth = None
best_samples_split = None
best_samples_leaf = None

start_time = time.perf_counter()

for depth in range(1,15):
    for split in range(2,10):
        for leaf in range(1,10):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=split, min_samples_leaf=leaf)
            model.fit(features_train, target_train)
            answer = model.predict(features_validate)
            accuracy = accuracy_score(answer, target_validate)
            if accuracy > best_tree_accuracy:
                best_tree_accuracy = accuracy
                best_tree_depth = depth
                best_samples_split = split
                best_samples_leaf = leaf

finish_time = time.perf_counter()                

tree_study_time = finish_time - start_time

print(f'Лучшая точность наблюдается при глубине:{best_tree_depth}')
print(f'Лучшая точность наблюдается при минимальном кол-ве выборок для разделения: {best_samples_split}')
print(f'Лушчая точность наблюдается при минимальном кол-ве образцов в одном листе: {best_samples_leaf}')
print(f'Лучшая точность решающего дерева: {best_tree_accuracy}')
print()
print(f'Затраченное время на обучения наилучшего решаюшего дерева: {tree_study_time} секунд')

Лучшая точность наблюдается при глубине:11
Лучшая точность наблюдается при минимальном кол-ве выборок для разделения: 2
Лушчая точность наблюдается при минимальном кол-ве образцов в одном листе: 8
Лучшая точность решающего дерева: 0.8087091757387247

Затраченное время на обучения наилучшего решаюшего дерева: 6.320871017873287 секунд


Приятно заметить, что лучшими оказались некие средние параметры, а не крайние - как я понимаю это значит, что мы попали в экстремум между недообучением и переобучением, получив максимальную точность

### Случайный лес:

Случайный лес по сути зависит от того, сколько деревьев он выращивает для принятия решения какие ответы взять за лучшие. Тут так же, как я понял, работает логика, что таких деревьев не должно быть слишком много для переобучения, но и не слишком мало для недообучения. Так что как раз сдеалем число деревьев изменяемым параметров в цикле

In [10]:
best_forest_accuracy = 0
best_tree_number = None

start_time = time.perf_counter()

for tree in range(1,10):
    model = RandomForestClassifier(random_state=12345, n_estimators=tree)
    model.fit(features_train, target_train)
    answer = model.predict(features_validate)
    accuracy = accuracy_score(answer, target_validate)
    if accuracy > best_forest_accuracy:
        best_forest_accuracy = accuracy
        best_tree_number = tree

finish_time = time.perf_counter()                

forest_study_time = finish_time - start_time

print(f'Лучшая точность наблюдается при числе деревьев: {best_tree_number}')
print(f'Лучшая точность случайного леса: {best_forest_accuracy}')
print()
print(f'Затраченное время на обучения случайного леса: {forest_study_time} секунд')

Лучшая точность наблюдается при числе деревьев: 4
Лучшая точность случайного леса: 0.7884914463452566

Затраченное время на обучения случайного леса: 0.2093927264213562 секунд


Оказалось, что лес деревьев в данной ситуации выучился гораздо быстрее, чем дерево решений. Думаю это связано с тем, что датафрейм в принципе не очень большой, и таким образом выходит гораздо дольше пебирать комбинации гиперпараметров решающего дерева по вложенным циклам для выбора наилучшей его модели, чем прогнать всего 1 цикл с выбором оптимального числа деревьев для случайного леса, который на небольшом датафрейме учится почти с той же скоростью, что и обычное дерево решений

### Логистическая регрессия

Для логистической регрессии из теоретической части и из свободных источников я не очень понял, какими гиперпараметрами вообще можно оперировать и вообще стоит ли с точки зрения здравого смысла в них копаться, но если такие есть, то буду рад если подскажешь их краткое описание и что конкретно они меняют

In [11]:
start_time = time.perf_counter()

model = LogisticRegression(random_state=12345, solver='lbfgs')
# не очень уверен что такое solver, но без него вылезало предупреждение, что его по-хорошему надо указать как lbfgs
model.fit(features_train, target_train)
answer = model.predict(features_validate)
accuracy = accuracy_score(answer, target_validate)

finish_time = time.perf_counter()

logreg_study_time = finish_time - start_time

print(f'Точность логистической регрессии: {accuracy}')
print()
print(f'Затраченное время на обучения модели логистической регрессии: {logreg_study_time} секунд')

Точность логистической регрессии: 0.7013996889580093

Затраченное время на обучения модели логистической регрессии: 0.017170555889606476 секунд


**Общий вывод по сравнению моделей:** Оказалось, что у логистической регрессии в данном случае самая низкая точность, но при этом сама быстрая скорость обучения (естественно вероятно потому, что не было никакого цикла с изменением гиперпараметров) ; у случайного леса весьма быстрая скорость обучения и при этом весьма высокая точность; у дерева решений вероятно всего при незменных гиперпараметрах была бы точнсть ниже, чем у случайного леса, но из-за того что мы потратили достаточного много времени на их перебор через вложенные циклы, то удалось добиться такой комбинации, что она даже перебила лес деревьев. Попроуем использовать именно эту модель для проверки точности по тестовой выборке

## Проверьте модель на тестовой выборке

Ещё раз выведем какие гиперпараметры были выбраны, как наилучшие для дерева решений:

In [12]:
print(f'Лучшая точность наблюдается при глубине:{best_tree_depth}')
print(f'Лучшая точность наблюдается при минимальном кол-ве выборок для разделения: {best_samples_split}')
print(f'Лушчая точность наблюдается при минимальном кол-ве образцов в одном листе: {best_samples_leaf}')

Лучшая точность наблюдается при глубине:11
Лучшая точность наблюдается при минимальном кол-ве выборок для разделения: 2
Лушчая точность наблюдается при минимальном кол-ве образцов в одном листе: 8


In [13]:
best_model = DecisionTreeClassifier(random_state=12345, 
                                    max_depth=best_tree_depth, 
                                    min_samples_split=best_samples_split, 
                                    min_samples_leaf=best_samples_leaf)

In [14]:
best_model.fit(features_train, target_train)
answer = best_model.predict(features_validate)
accuracy = accuracy_score(answer, target_validate)
print(f'Вычисленная ранее точность модели на валидационной выборке: {accuracy}')

Вычисленная ранее точность модели на валидационной выборке: 0.8087091757387247


Теперь проверим, какую точность модель покажет на тестовой выборке:

In [15]:
answer = best_model.predict(features_test)
accuracy = accuracy_score(answer, target_test)
print(f'Точность модели на тестовой выборке: {accuracy}')

Точность модели на тестовой выборке: 0.80248833592535


Требуемая точность ответов была заявлена, как 0.75, соответственно можем сказать, что всё выполнено успешно:

In [16]:
required_accuracy = 0.75
if accuracy >= required_accuracy:
    print('модель работает достаточно точно')
else:
    print('модель не выдаёт нужной точности')

модель работает достаточно точно


## (бонус) Проверьте модели на адекватность

Насколько я понял из теории, проверка модели на адекватность - это проверка на то, насколько она вообще работает лучше, чем в лоб записанные предсказания без какого-либо машинного обучения. Если модель оказывается даже худшим предсказателем, то по сути она невменяема и не несёт никакой пользы. Попробуем при помощи библиотеки random просто записать ряд 0 и 1 и сравнить насколько высокую точность покажет такое предсказание, чтобы знать, ниже какой точности модели в данном проекте можно считать невменяемыми

In [17]:
random_predictions = []
for number in range(len(target_test)):
    random_predictions.append(random.randint(0,1))

In [18]:
random_accuracy = accuracy_score(pd.Series(random_predictions), target_test)

In [19]:
random_accuracy

0.5023328149300156

В целом было ожидаемо, что когда мы говорим о предсказании ряда из 0 и 1, то просто на рандоме можно его угадывать примерно с вероятностью 50 на 50. Все выше обученные модели имели точность предсказаний в диапазоне 0.7 - 0.8, так что могут считаться адекватными, но при определённых неудачных параметрах (например слишком большой глубине дерева или слишком большом размере леса деревьев при переобучении, или наоборот их малом числе при недообучении, полученное значение **random_accuracy** может служить своеобразным фильтром чтобы сразу отказаться от модели, так как даже рандомный генератор чисел работает эффективнее

In [20]:
from sklearn.dummy import DummyClassifier

Сначала вручную проверим с какой точностью работают модели, где предсказаны просто 0 или 1, так как по сути это 2 единственно возможных предсказания в данной задаче.

In [21]:
zero_predictions = []
for number in range(len(target_test)):
    zero_predictions.append(0)
one_predictions = []
for number in range(len(target_test)):
    one_predictions.append(1)

In [22]:
zero_accuracy = accuracy_score(pd.Series(zero_predictions), target_test)
one_accuracy = accuracy_score(pd.Series(one_predictions), target_test)
print(f'Точность предсказаний из одних только нулей: {zero_accuracy}')
print(f'Точность предсказаний из одних только единиц: {one_accuracy}')

Точность предсказаний из одних только нулей: 0.6905132192846034
Точность предсказаний из одних только единиц: 0.3094867807153966


Очевидно чаще в тестовой выборке встречаются именно 0 в качестве правильных предсказаний, то есть большинство использует тариф smart. Попробуем заставить об этом догадаться и "глупую модель" посмотрев, что она выберет

In [23]:
dummy_clf = DummyClassifier(strategy="most_frequent") 
# модель решит, что резоннее всего выбрать самый часто встречающийся ответ в качестве верного предсказания

In [24]:
dummy_clf.fit(features_train, target_train)
answer = dummy_clf.predict(features_test)
accuracy = accuracy_score(answer, target_test)
print(f'Точность "глупой" модели на тестовой выборке: {accuracy}')

Точность "глупой" модели на тестовой выборке: 0.6905132192846034


In [25]:
answer

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Собственно, как я понимаю всё вышло по плану: "глупая модель" догадалась, что чаще всего встречается именно 0 в качестве правильного ответа в тренировочной выборке, и решила просто всегда отвечать 0 в качестве предсказаний и на тестовой. В целом, точность почти в 0.7 в данном случае вполне конкурентна даже на фоне полученных выше обученных моделей, поэтому оказалось весьма интересно. Спасибо что дал ссылку ознакомиться с такими моделями